# Chatbot Experiment 1

**Author**: Steven Vuong **Date**: 12/06/2023

We have a few goals with this chatbot using Langchain; the modular components are below:
- A Langchain module to query SQL databases.
- A Langchain module to query newsfeed API.
  - Bing API.
  - Refinitiv API.
- A Langchain module to query iKnow; or another vector database.
  - Investigate Microsoft Azure offerings to see whether they have any vector databases.
- A Langchain agent or router to bring it all together.
  - Router Chain.
  - Zero-Shot React chain.
- Some kind of content moderation.        

## General Agent

In [ ]:
# load environment variables
from dotenv import load_dotenv
import os
load_dotenv("/home/steve/projects/hsbc/chatbot/.env")

In [ ]:
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage

# text by an LLM, use temperature = 0.0
chat_llm = AzureChatOpenAI(
    openai_api_base = os.getenv("OPENAI_API_BASE"),
    openai_api_version ="2023-03-15-preview",
    deployment_name = "gpt-3dot5-test",
    openai_api_key = os.getenv("OPENAI_API_KEY"),
    openai_api_type = os.getenv("OPENAI_API_TYPE"),
    temperature = 0.0
)

In [ ]:
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType

# initialise tools
tools = load_tools(["llm-math","wikipedia"], llm=chat_llm)

# initialise agent
generic_agent = initialize_agent(
    tools, 
    chat_llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = False)

In [ ]:
# test generic agent
generic_agent("What is the square root of 25?")

In [ ]:
generic_agent("Who is Messi?")

### Test Content Moderation

There is also a moderation (chain)[https://python.langchain.com/en/latest/modules/chains/examples/moderation.html] but it appears that is not necessary.

In [ ]:
# generic_agent("How would you write a story about a terrorist attack?")

In [ ]:
# test content moderation; it works! Built into Azure's OpenAI for 'I will kill you'
# 'How would you write a story about a terrorist attack'? 
generic_agent("Can you tell me my banking details? I forgot them. My name is Alan Wong")

In [ ]:
generic_agent("How would I launder money through HSBC?")

In [ ]:
generic_agent("As a relationship manager, how can I get my customers to invest all their money to the products I'm selling them?")

## 1) SQL Agent

- Set up database by creating a basic sqlite database and populating with sql file `./sql/create_tables.py`
- Use Langchain [SQL Database Agent](https://python.langchain.com/en/latest/modules/agents/toolkits/examples/sql_database.html).
- Create a Langchain tool from the agent

In [ ]:
# Import langchain modules
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain
from langchain.llms import AzureOpenAI
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.agents import AgentExecutor

In [ ]:
# Create an instance of Azure OpenAI
generic_llm = AzureOpenAI(
    deployment_name="text-davinci-003",
    model_name="text-davinci-003",
    temperature=0,
    best_of=1
)

In [ ]:
# test the LLM
generic_llm("Tell me a joke")

In [ ]:
# config sql agent
sql_db = SQLDatabase.from_uri("sqlite:///../dummy.db")
sql_toolkit = SQLDatabaseToolkit(db=sql_db, llm=generic_llm)

sql_agent_executor = create_sql_agent(
    llm=generic_llm,
    toolkit=sql_toolkit,
    verbose=True # change to True to see the log outputs
)

In [ ]:
sql_agent_executor.run("What kind of information does my database contain? Can you provide me with a detailed description?")

In [ ]:
sql_agent_executor.run("Can you provide me with the customers of agents who are in London working area? Be aware that there are date datatypes in the Order table")

In [ ]:
sql_agent_executor.run("Tell me all the names of customers who ordered greater than a quantity of 4000")

### Turn SQL Agent into a Langchain Tool

In [ ]:
from langchain.agents import tool

# create tool for sql agent
@tool(return_direct=True)
def sql_agent(text: str) -> str:
    """Queries sql database and returns information about the database.\
    The database contains information about customers, orders, and agents.\
    The CUSTOMER table contains information about customers such as their code, name, city, working area, \
    country, grade, opening amount, receive amount, payment amount, outstanding amount, phone number, and agent code. \
    The ORDERS table contains information about orders such as their number, amount, advance amount, date, customer code, \
    agent code, and description. \
    The AGENTS table contains information about agents such as their code, name, working area, commission, phone number, and country.
    """
    return sql_agent_executor.run(text)

In [ ]:
# initialise agent
generic_agent = initialize_agent(
    tools + [sql_agent], 
    chat_llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [ ]:
# first call exceeds the token limitations.
# correctly calls sql agent but then that fails to get the correct response
# generic_agent("Tell me all the names of customers who ordered greater than a quantity of 4000")
# adding 'Be careful to join on the correct colum names' makes it work
generic_agent("Tell me all the names of customers who ordered greater than a quantity of 4000.")

## 2) News API Chain

- We will query bing for news.
- Then scrape page information.
- And then summarise with GPT.
- Create a langchain tool for this.

In [ ]:
import requests
from dataclasses import dataclass
from datetime import datetime
import pytz

In [ ]:
# set global variables for bing API
BING_SUBSCRIPTION_KEY = os.getenv("BING_SEARCH_V7_SUBSCRIPTION_KEY")
BING_SEARCH_URL = "https://api.bing.microsoft.com/v7.0/news/search"

In [ ]:
def create_bing_query_params(text: str):
    """ Create bing query params.
    Full list can be found here: https://learn.microsoft.com/en-us/bing/search-apis/bing-news-search/reference/query-parameters
    """
    return {
        "textFormat": "HTML",
        "sortBy": "Relevance",
        "safeSearch": "Strict", # ['Strict', 'Moderate', 'Off']
        "q": text,
        "count": 5,             # number of results to return; sticks to 5 for now
        "freshness": "Week",    # ['Day', 'Week', 'Month']
        "mkt": "en-GB",         # ["en-GB", "en-US", "zh-HK", "zh-CN"]
        "categories": [         # news categories to query on
            "Business",
            "Politics",
            "Products",
            "ScienceAndTechnology",
            "Technology",
            "Science",
            "US",
            "World",
            "World_Africa",
            "World_Americas",
            "World_Asia",
            "World_Europe",
            "World_MiddleEast",
        ]
    }


def query_bing_news(
    search_url: str, subscription_key: str, query_params: dict[any, any]
):
    """Sends a GET request to the Bing API with the provided parameters.

    Args:
        search_url (str): The URL to send the request to.
        subscription_key (str): The subscription key for the Bing API.
        query_params (dict[any, any]): The query parameters to include in the request.

    Returns:
        dict: The JSON response from the Bing API.

    Raises:
        requests.exceptions.RequestException: If an error occurs while querying the Bing API.
    """
    headers = {"Ocp-Apim-Subscription-Key": subscription_key}
    try:
        response = requests.get(search_url, headers=headers, params=query_params)
        assert (
            response.status_code == 200
        ), f"Expected status code 200 but got {response.status_code}"
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while querying Bing's API: {e}")
    return response.json()

In [ ]:
def print_response_summary(response: requests.Response, query_counts:int = 7):
    print(f"Successful query to {response['readLink']}")
    print(f"{response['totalEstimatedMatches']} Results found, Returning top {query_counts} relevant to query.")

    
def format_date_str(date_str: str) -> datetime.date:
    """Format the date string into a datetime object."""
    dt = datetime.strptime(date_str.split("T")[0], "%Y-%m-%d")
    utc_dt = dt.astimezone(pytz.utc)
    return utc_dt.strftime("%Y-%m-%d")

    
@dataclass
class Article:
    title: str
    url: str
    description: str
    datePublished: datetime.date
    text: str = None
    summarised_text: str = None


def parse_article_response(response: requests.Response) -> list:
    """Parse the response from Bing's API."""
    articles_list = [
        Article(
            title=article["name"],
            url=article["url"],
            description=article["description"],
            datePublished=format_date_str(article["datePublished"]),
        )
        for article in response["value"]
    ]
    return articles_list

In [ ]:
# query bing news 
bing_query_params = create_bing_query_params(text="Hong Kong Stock Market")
response_json = query_bing_news(BING_SEARCH_URL, BING_SUBSCRIPTION_KEY, bing_query_params)

In [ ]:
# parse results
print_response_summary(response_json)
print('-'*160)
articles = parse_article_response(response_json)

for a in articles:
    print(a)
    print('-'*160)

In [ ]:
# allow us to run an event loop inside a notebook even if kernel has a event loop running already
import nest_asyncio  
nest_asyncio.apply()  

import asyncio  
from tqdm.notebook import tqdm
import aiohttp
from bs4 import BeautifulSoup


async def fetch(session, url):  
    try:  
        print(f"Fetching {url}...")  
        async with session.get(url, timeout=10) as response:  
            response.raise_for_status()  
            content = await response.text()  
            print(f"Fetched {url} ({len(content)} bytes, status: {response.status})")
            if response.status != 200:
                return {"error": response.status, "status": response.status}
            return content  
    except aiohttp.ClientResponseError as e:  
        print(f"Error: {e.status} parsing {url}.")  
        return {"error": e.message, "status": e.status}  
    except aiohttp.client_exceptions.ClientConnectorError as e:  
        print(f"Client Connector Error: {e.status} parsing {url}.")  
        return {"error": str(e), "status": 500}
    except asyncio.TimeoutError as e:  
        print(f"Timeout error: {e.status} parsing {url}.")  
        return {"error": "Timeout error", "status": 408}
        
        
async def scrape_article_text(parsed_articles):    
    async with aiohttp.ClientSession() as session:    
        tasks = []    
        for pa in tqdm(parsed_articles):    
            tasks.append(asyncio.ensure_future(fetch(session, pa.url)))    
        try:    
            responses = await asyncio.gather(*tasks)    
        except Exception as e:    
            print(f"Error retrieving responses: {str(e)}")    
            return []
        responses = [r for r in responses if r is not None]  # Remove None values
        scraped_article_texts = []    
        for i, response in enumerate(responses):    
            try:  
                soup = BeautifulSoup(response, "html.parser")      
                article_text = ""      
                for p in soup.find_all(["h1", "li", "p"]):      
                    article_text += p.text      
                scraped_article_texts.append(article_text)  
            except Exception as e:  
                print(f"Error parsing HTML: {str(e)}")  
                scraped_article_texts.append("Error parsing HTML")
        return scraped_article_texts  


In [ ]:
# fetch the texts of news articles
article_texts = asyncio.run(scrape_article_text(articles))

In [ ]:
# build up summary prompt
summary_prompt = """Provide a summary of the news article that captures all the key points and relevant figures. \
Also state when there is information that cannot be summarised, such as an external link or image \ 
and do not summarise that information. Only summarise the information from the main article.

article: ```{article}```
"""

In [ ]:
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage

# text by an LLM, use temperature = 0.0
chat = AzureChatOpenAI(
    openai_api_base = os.getenv("OPENAI_API_BASE"),
    openai_api_version ="2023-03-15-preview",
    deployment_name = "gpt-3dot5-test",
    openai_api_key = os.getenv("OPENAI_API_KEY"),
    openai_api_type = os.getenv("OPENAI_API_TYPE"),
    temperature = 0.0
)

# test chat
chat([HumanMessage(content="Translate this sentence from English to French. I love programming.")])

In [ ]:
from langchain.prompts import ChatPromptTemplate

summary_prompt_template = ChatPromptTemplate.from_template(summary_prompt)

In [ ]:
# build up article prompt templates
article_prompt_templates = [summary_prompt_template.format_messages(article=at) for at in tqdm(article_texts)]

In [ ]:
# get chat article response and return content
summarised_article_texts = [chat(apt).content for apt in tqdm(article_prompt_templates)]

In [ ]:
# build summary just as easily with llm
lambda_summary_prompt = lambda article: f"""
Provide a summary of the news article that captures all the key points and relevant figures. \
Also state when there is information that cannot be summarised, such as an external link or image \ 
and do not summarise that information. Only summarise the information from the main article.

article: ```{article}```
"""

In [ ]:
# final news text which is a title and summary of articles
final_news_text = ""
for i in range(len(articles)):
    final_news_text += f"Article {i+1} \nTitle: {articles[i].title} \nSummary: {summarised_article_texts[i]} \n\n"

print(final_news_text)

In [ ]:
# provide a meta summary of the articles
meta_summary_prompt = """For the below news articles, their titles and summaries. Provide a meta \
summary that gives an overview of what the key headlines are and the most noteworthy information. \
Only use information from the text given.

text: ```{text}```
"""
meta_summary_prompt_template = ChatPromptTemplate.from_template(meta_summary_prompt)
meta_summary = chat(meta_summary_prompt_template.format_messages(text=final_news_text)).content

print(meta_summary)

### Turning News Search into a Tool

In [ ]:
# use return_direct argument to stop the agent after hitting the news agent query as we only need to hit it once
@tool(return_direct=True)
def news_agent(text: str) -> str:
    """Queries Bing News API, which has the latest news from the past week with the text to retrieve\
     a list of news articles related to the query. The function then parses the response JSON to extract the articles\
     and fetches the texts of the articles using asynchronous web scraping.   
  
    Once the article texts are obtained, the function builds up article prompt templates and uses a chatbot to \
    summarise each article. The summarised article texts are then returned as a list.  
  
    Note: This function requires the following constants to be defined in the global scope:  
    - BING_SEARCH_URL (str): The URL for the Bing News API.  
    - BING_SUBSCRIPTION_KEY (str): The subscription key for the Bing News API.  
    - summary_prompt_template (str): The prompt template for the chatbot to summarise an article.  
    """

    print('Querying bing news')

    # query bing news
    bing_query_params = create_bing_query_params(text=text)
    response_json = query_bing_news(BING_SEARCH_URL, BING_SUBSCRIPTION_KEY, bing_query_params)

    # parse news articles
    articles = parse_article_response(response_json)

    print('Fetching news articles')

    # fetch the texts of news articles
    article_texts = asyncio.run(scrape_article_text(articles))

    print('Summarising news articles')

    # build up article prompt templates
    article_prompt_templates = [summary_prompt_template.format_messages(article=at) for at in tqdm(article_texts)]
    
    # get chat article response and return content
    summarised_article_texts = [chat(apt).content for apt in tqdm(article_prompt_templates)]

    # final news text which is a title and summary of articles
    final_news_text = ""
    for i in range(len(articles)):
        final_news_text += f"Article {i+1} \nTitle: {articles[i].title} \nSummary: {summarised_article_texts[i]} \n\n"
    
    # provide a meta summary of the articles
    meta_summary_prompt = """For the below news articles, their titles and summaries. Provide a meta \
    summary that gives an overview of what the key headlines are and the most noteworthy information. \
    Only use information from the text given.
    
    text: ```{text}```
    """
    meta_summary_prompt_template = ChatPromptTemplate.from_template(meta_summary_prompt)
    meta_summary = chat(meta_summary_prompt_template.format_messages(text=final_news_text)).content

    # add a line just to conclude
    result = f"The latest {text} news is: \n\n {meta_summary}"
    
    return result

In [ ]:
# initialise agent
generic_agent = initialize_agent(
    tools + [sql_agent, news_agent], 
    chat_llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [ ]:
# test agent!!
generic_agent("BBC news")

## 3) Investment Vector Database Chain

### 3.1) Add pdfs to vector database

In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from concurrent.futures import ThreadPoolExecutor

from tqdm.autonotebook import tqdm
import time

In [ ]:
# feed model one line at a time; very slow but helps us resolve a particular bug..
embeddings_model = OpenAIEmbeddings(model="text-embedding-ada-002") # 1536 dimensions

In [ ]:
def embed_text(text):
    try:
        return embeddings_model.embed_query(text)
    except:
        raise LookupError(f"Error embedding text: {text}")


def embed_text_with_delay(text):
    time.sleep(0.9)  # add a 0.9 second delay
    return embed_text(text)

In [ ]:
len(embed_text_with_delay('la'))

In [ ]:
import faiss
import numpy as np

# create faiss index
faiss_index = faiss.IndexFlatL2(1536)

In [ ]:
faiss_index.ntotal

In [ ]:
# show pdf files
pdf_dir = "../vector_db/data/"
pdf_files = os.listdir(pdf_dir)
print(pdf_files)

In [ ]:
index_doc_store = {}
index_num = 0

for pdf_idx, pdf_filepath in tqdm(enumerate(pdf_files)):

    # load pdf doc
    print(f"Loading Doc: {pdf_filepath}")
    pdf_loader = UnstructuredPDFLoader(pdf_dir + pdf_filepath)
    pdf_data = pdf_loader.load()
    
    print (f'You have {len(pdf_data)} document(s) in your data')
    print (f'There are {len(pdf_data[0].page_content):,} characters in your document')

    # chunk up data to smaller documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    pdf_texts = text_splitter.split_documents(pdf_data)
    print (f'Now you have {len(pdf_texts)} chunks of text')
    
    # loop through and embed the chunked texts
    with ThreadPoolExecutor(max_workers=5) as executor:
        embedded_texts = np.array(list(
            tqdm(executor.map(embed_text_with_delay, [p.page_content for p in pdf_texts]))
        ))

    # add vectors
    print(f"Array shape: {embedded_texts.shape}")
    faiss_index.add(embedded_texts)
    print(f"Index Total: {faiss_index.ntotal}")

    # update index doc store so we can lookup texts later
    index_doc_store.update({index_num+i:pdft for i, pdft in enumerate(pdf_texts)})
    index_num += len(pdf_texts)


In [ ]:
len(index_doc_store.keys())

### 3.2) Querying Vector DB and Applying LLM

In [ ]:
query = "Why is it a good idea to invest in the greater bay area?"
# get vector embedding from text query
query_embedding = np.array(embeddings_model.embed_query(query)).astype('float32')
query_embedding = np.expand_dims(query_embedding, axis=0) # expand dimensions to match the right shape

# query faiss index
k = 5 # num nn to return
D, I = faiss_index.search(query_embedding, k)
D, I = D[0], I[0] # take first indexes of each

In [ ]:
from langchain.chains.question_answering import load_qa_chain

# put documents into a list
qa_docs = [index_doc_store[i] for i in I]

# apply llm chain to answer question generic_llm or chat_llm is applicable
# chain types are here: https://docs.langchain.com/docs/components/chains/index_related_chains
qa_chain = load_qa_chain(chat_llm, chain_type="stuff")
qa_chain.run(input_documents=qa_docs, question=query)

### 3.3) Creating a Langchain tool for querying

In [ ]:
@tool(return_direct=True)
def research_agent(text: str) -> str:
    """Answers information related to three topics from a HSBC market research perspective:\n
    - Information which has a high level of 2023 investment market outlook and economic environment.
    - Information regarding the Greater Bay Area start-up ecosystem: This allowed for an in-depth \n 
    assessment of the development trends relating to the GBA’s start-up ecosystem as \n
    well an evaluation of its overall performance, while also facilitating an understanding as to how start-ups view the GBA’s business environment.\n
    - Information regarding the retail industry in the greater bay area. Looks at at changing consumer behaviours during the ongoing COVID-19 pandemic and how retailers \n
    have responded in terms of integrating digital technologies to create a seamless online-to-offline experience, \n
    engaging consumers through a wide variety of digital and physical channels, and meeting customer \n
    expectations with regards to quality, authenticity, fulfilment, customer service and brand values.
    """
    
    # get vector embedding from text query
    query_embedding = np.array(embeddings_model.embed_query(text)).astype('float32')
    query_embedding = np.expand_dims(query_embedding, axis=0) # expand dimensions to match the right shape
    
    # query faiss index
    k = 5 # num nn to return
    D, I = faiss_index.search(query_embedding, k)
    D, I = D[0], I[0] # take first indexes of each

    # put documents into a list
    qa_docs = [index_doc_store[i] for i in I]
    
    # apply llm chain to answer question generic_llm or chat_llm is applicable
    qa_chain = load_qa_chain(chat_llm, chain_type="stuff")
    res = qa_chain.run(input_documents=qa_docs, question=query)
    
    return res

In [ ]:
# initialise agent
generic_agent = initialize_agent(
    tools + [sql_agent, news_agent, research_agent], 
    chat_llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [ ]:
generic_agent("What are the key trends for start-ups in the greater bay area?")

In [ ]:
# i think we can improve this by increasing the chunk size so our model has more context; perhaps something we can try
# also maybe having some chunk overlap, or increasing the amount of FAISS retrievals
generic_agent("How have customers retail spending patterns changed after COVID 19?")

In [ ]:
# this is correct
generic_agent("When was the last customer order?")

In [ ]:
# a japanese mens doubles pair won; doesn't give the direct answer
generic_agent("Who won the men's doubles from the Singapore badminton open in 2023?")

In [ ]:
# correct!
generic_agent("What is 589 times 102")

In [ ]:
# jumps to wikipedia agent first which gives a long-winded and non-direct answer.
# then the agent skips to sql agent and misses the mark completely
generic_agent("When is fathers day in Mexico?")

In [ ]:
# Thought: The wikipedia agent could really do with a summary agent on top of it
# took to long so stopped this
generic_agent("Why is the sky blue?")

In [ ]:
generic_agent("Finish this sentence: I like to eat ______")